In [1]:
import tweepy
import pandas as pd
import numpy as np
import re

from authorization_keys import consumer_key, consumer_secret, token, token_key # User specific, acquired from Twitter 

In [2]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)

auth.set_access_token(token, token_key)

In [3]:
api = tweepy.API(auth)

In [4]:
tweets = []
for tweet in tweepy.Cursor(api.search, q='MTBPS2019 -filter:retweets', tweet_mode='extended').items(3000):
    tweets.append(tweet)

In [5]:
len(tweets)

2293

Looks like 2293 is the most number of tweets MTBPS2019 has.

In [138]:
# Just checking how much data is in a single tweet
tweets[0]

Status(_api=<tweepy.api.API object at 0x000001D454BDCE80>, _json={'created_at': 'Thu Nov 07 11:04:45 +0000 2019', 'id': 1192397471307186179, 'id_str': '1192397471307186179', 'full_text': 'Post #MTBPS2019 and the release of the second version of \n@TreasuryRSA’s economic paper, we’re faced with many questions - most importantly those of congruency and validity. \n\nWe explore the ⁉ in our #ScienceOfModelling series. \n\nKey reading 👉https://t.co/24ARc1NLwM', 'truncated': False, 'display_text_range': [0, 266], 'entities': {'hashtags': [{'text': 'MTBPS2019', 'indices': [5, 15]}, {'text': 'ScienceOfModelling', 'indices': [200, 219]}], 'symbols': [], 'user_mentions': [{'screen_name': 'TreasuryRSA', 'name': 'National Treasury', 'id': 1484243208, 'id_str': '1484243208', 'indices': [58, 70]}], 'urls': [{'url': 'https://t.co/24ARc1NLwM', 'expanded_url': 'https://bit.ly/2NXL5Wj', 'display_url': 'bit.ly/2NXL5Wj', 'indices': [243, 266]}]}, 'metadata': {'iso_language_code': 'en', 'result_type': 're

In [139]:
# Mensioned user in the tweet
tweets[0].entities['user_mentions'][0]['screen_name']

'TreasuryRSA'

In [85]:
df = pd.DataFrame()

In [86]:
def clean_tweet(tweet):
        return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)"," ", tweet).split())

In [112]:
df['date'] = [twit.created_at for twit in tweets]
df['id'] = [twit.user.name for twit in tweets]
df['text'] = [clean_tweet(twit.full_text) for twit in tweets]
df['target'] = [twit.entities['user_mentions'] if twit.entities['user_mentions'] else None for twit in tweets]

In [128]:
df.head()

,date,id,text,target
0,2019-11-07 11:04:45,ADRS,Post MTBPS2019 and the release of the second v...,"[{'screen_name': 'TreasuryRSA', 'name': 'Natio..."
1,2019-11-07 10:25:08,MBI Matseke Business Investment (PTY)LTD,Recap of mtbps2019,None
2,2019-11-07 10:09:35,Institute for Economic Justice,Some say austerity will make things worse some...,"[{'screen_name': 'budgetjusticesa', 'name': 'B..."
3,2019-11-07 09:13:42,Equal Education,READ State spending cuts higher taxes doing mo...,None
4,2019-11-07 06:40:01,CapeTalk on 567AM,It would be irresponsible if the Reserve Bank ...,None


Need to sort out the target column so that it can only have a list of targets

In [129]:
def mentioned_names(target_list):
    m = [] # List of names targeted
    for i in range(len(target_list)):
        m.append(target_list[i]['name'])
    return m

In [131]:
df['target'] = [mentioned_names(x) if x is not None else None for x in df['target']]

In [149]:
df.head()

,date,id,text,target
0,2019-11-07 11:04:45,ADRS,Post MTBPS2019 and the release of the second v...,[National Treasury]
1,2019-11-07 10:25:08,MBI Matseke Business Investment (PTY)LTD,Recap of mtbps2019,None
2,2019-11-07 10:09:35,Institute for Economic Justice,Some say austerity will make things worse some...,[Budget Justice Coalition SA]
3,2019-11-07 09:13:42,Equal Education,READ State spending cuts higher taxes doing mo...,None
4,2019-11-07 06:40:01,CapeTalk on 567AM,It would be irresponsible if the Reserve Bank ...,None


In [135]:
# The mensioned names on the 100th tweet on the list
df['target'].iloc[100]

['Tito Mboweni', 'Busi Mavuso']

In [141]:
# Who the 100th tweet was from
df['id'].iloc[100]

'Business Leadership South Africa'

In [142]:
# What the tweet said
df['text'].iloc[100]

'Sobriety is the main message in the mboweni MTBPS with alarming expectations of the debt GDP ratio reaching 81 says BLSA s MTBPS2019 See'

In [136]:
df['target'].iloc[100][0]

'Tito Mboweni'

In [137]:
df.to_pickle('tweets.pkl')

In [140]:
df['id'].nunique()

615

In [145]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2293 entries, 0 to 2292
Data columns (total 4 columns):
date      2293 non-null datetime64[ns]
id        2293 non-null object
text      2293 non-null object
target    766 non-null object
dtypes: datetime64[ns](1), object(3)
memory usage: 71.8+ KB
